In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image
import os
from google.colab import drive
drive.mount('/content/drive')

# Configurations
IMG_HEIGHT, IMG_WIDTH = 32, 64  # Resized image size for training
NUM_CLASSES = 36  # Assuming digits (0-9) + uppercase letters (A-Z)

# Define Deeper CNN Model
class DeeperCharRecogCNN(nn.Module):
    def __init__(self, num_classes):
        super(DeeperCharRecogCNN, self).__init__()

        # Convolutional Layer 1
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # Change input channels to 3 for RGB
        self.pool1 = nn.MaxPool2d(2, 2)  # Max pooling

        # Convolutional Layer 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 64 filters
        self.pool2 = nn.MaxPool2d(2, 2)  # Max pooling

        # Convolutional Layer 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # 128 filters
        self.pool3 = nn.MaxPool2d(2, 2)  # Max pooling

        # Convolutional Layer 4
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)  # 256 filters
        self.pool4 = nn.MaxPool2d(2, 2)  # Max pooling

        # Fully connected layers
        self.fc1 = nn.Linear(256 * 2 * 4, 1024)  # Adjust input size based on final feature map
        self.fc2 = nn.Linear(1024, 512)  # Intermediate fully connected layer
        self.fc3 = nn.Linear(512, num_classes)  # Final output layer

        # Dropout layers to reduce overfitting
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))  # Conv1 -> ReLU -> Pooling
        x = self.pool2(F.relu(self.conv2(x)))  # Conv2 -> ReLU -> Pooling
        x = self.pool3(F.relu(self.conv3(x)))  # Conv3 -> ReLU -> Pooling
        x = self.pool4(F.relu(self.conv4(x)))  # Conv4 -> ReLU -> Pooling

        x = x.view(-1, 256 * 2 * 4)  # Flatten the output for fully connected layers
        x = F.relu(self.fc1(x))  # Fully connected layer 1
        x = self.dropout(x)  # Apply dropout to reduce overfitting
        x = F.relu(self.fc2(x))  # Fully connected layer 2
        x = self.fc3(x)  # Output layer (classification)

        return x

# Dataset class (for loading character images)
class CharacterDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Traverse the dataset directory and collect image paths and their corresponding labels
        for label, folder in enumerate(os.listdir(root_dir)):
            folder_path = os.path.join(root_dir, folder)
            if os.path.isdir(folder_path):
                for img_name in os.listdir(folder_path):
                    img_path = os.path.join(folder_path, img_name)
                    if img_path.endswith(".png") or img_path.endswith(".jpg"):  # Filter for images
                        self.image_paths.append(img_path)
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # Open the image
        image = Image.open(img_path).convert("RGB")

        # Apply the transformations if provided
        if self.transform:
            image = self.transform(image)

        return image, label

# Define the data transformations (resizing and normalization)
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),  # Resize all images to 32x64
    transforms.ToTensor(),       # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize
])

# Define the root directory where the dataset is stored
root_dir = '/content/drive/My Drive/train_char/'

# Initialize the deeper model and move it to GPU
model = DeeperCharRecogCNN(num_classes=NUM_CLASSES)#.cuda()  # Moving model to GPU
print(model)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
num_epochs = 20
train_loader = DataLoader(CharacterDataset(root_dir, transform), batch_size=32, shuffle=True)

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for inputs, labels in train_loader:
        # No need to move tensors to GPU
        # If you're using a CPU, you can leave them as they are

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


# Save the model
model_save_path = "/content/drive/My Drive/DeepCharRecogCNN1.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved at {model_save_path}")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image
import os
from google.colab import drive

drive.mount('/content/drive')

# Configurations
IMG_HEIGHT, IMG_WIDTH = 28, 28  # Resized image size for training
NUM_CLASSES = 36  # Assuming digits (0-9) + uppercase letters (A-Z)

# Define Deeper CNN Model for Grayscale Images
class DeeperCharRecogCNN(nn.Module):
    def __init__(self, num_classes):
        super(DeeperCharRecogCNN, self).__init__()

        # Convolutional Layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2)

        # Compute the flattened size of the feature maps dynamically
        self._to_linear = None
        self._define_linear_input((1, IMG_HEIGHT, IMG_WIDTH))

        # Fully Connected Layers
        self.fc1 = nn.Linear(self._to_linear, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(p=0.5)

    def _define_linear_input(self, input_shape):
        with torch.no_grad():
            x = torch.zeros(1, *input_shape)
            x = self.pool1(F.relu(self.conv1(x)))
            x = self.pool2(F.relu(self.conv2(x)))
            x = self.pool3(F.relu(self.conv3(x)))
            x = self.pool4(F.relu(self.conv4(x)))
            self._to_linear = x.view(-1).shape[0]

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Dataset class (for loading character images)
class CharacterDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Traverse the dataset directory and collect image paths and their corresponding labels
        for label, folder in enumerate(os.listdir(root_dir)):
            folder_path = os.path.join(root_dir, folder)
            if os.path.isdir(folder_path):
                for img_name in os.listdir(folder_path):
                    img_path = os.path.join(folder_path, img_name)
                    if img_path.endswith(".png") or img_path.endswith(".jpg"):
                        self.image_paths.append(img_path)
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # Open the image as grayscale
        image = Image.open(img_path).convert("L")

        # Apply the transformations if provided
        if self.transform:
            image = self.transform(image)

        return image, label

# Define the data transformations (resizing and normalization for grayscale images)
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),  # Resize all images to 32x64
    transforms.ToTensor(),       # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize grayscale images
])

# Define the root directory where the dataset is stored
root_dir = '/content/drive/My Drive/train_char/'

# Initialize the deeper model
model = DeeperCharRecogCNN(num_classes=NUM_CLASSES)  # For grayscale images
print(model)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
num_epochs = 20
train_loader = DataLoader(CharacterDataset(root_dir, transform), batch_size=32, shuffle=True)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:


        optimizer.zero_grad()

        outputs = model(inputs)


        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Save the model
model_save_path = "/content/drive/My Drive/DeepCharRecogCNN_Grayscale.pt"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved at {model_save_path}")


Streaming output truncated to the last 5000 lines.
Labels shape: torch.Size([32])
Outputs shape: torch.Size([32, 36])
Input shape: torch.Size([32, 1, 28, 28])
Labels shape: torch.Size([32])
Outputs shape: torch.Size([32, 36])
Input shape: torch.Size([32, 1, 28, 28])
Labels shape: torch.Size([32])
Outputs shape: torch.Size([32, 36])
Input shape: torch.Size([32, 1, 28, 28])
Labels shape: torch.Size([32])
Outputs shape: torch.Size([32, 36])
Input shape: torch.Size([32, 1, 28, 28])
Labels shape: torch.Size([32])
Outputs shape: torch.Size([32, 36])
Input shape: torch.Size([32, 1, 28, 28])
Labels shape: torch.Size([32])
Outputs shape: torch.Size([32, 36])
Input shape: torch.Size([32, 1, 28, 28])
Labels shape: torch.Size([32])
Outputs shape: torch.Size([32, 36])
Input shape: torch.Size([32, 1, 28, 28])
Labels shape: torch.Size([32])
Outputs shape: torch.Size([32, 36])
Input shape: torch.Size([32, 1, 28, 28])
Labels shape: torch.Size([32])
Outputs shape: torch.Size([32, 36])
Input shape: torch